## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.74,77,100,18.41,overcast clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,62.69,80,88,13.35,overcast clouds
2,2,Atar,MR,20.5169,-13.0499,80.65,45,100,4.59,overcast clouds
3,3,Albany,US,42.6001,-73.9662,54.25,66,20,1.01,few clouds
4,4,Asosa,ET,10.0667,34.5333,61.47,81,9,4.54,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you (F) would like for your trip? "))

What is the minimum temperature (F) you would like for your trip? 70
What is the maximum temperature you (F) would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.74,77,100,18.41,overcast clouds
2,2,Atar,MR,20.5169,-13.0499,80.65,45,100,4.59,overcast clouds
9,9,Kudahuvadhoo,MV,2.6708,72.8944,81.88,76,99,10.58,overcast clouds
10,10,Yeppoon,AU,-23.1333,150.7333,78.58,75,46,5.86,scattered clouds
12,12,Aripuana,BR,-9.1667,-60.6333,86.31,55,98,0.16,overcast clouds
13,13,Vanimo,PG,-2.6741,141.3028,77.90,89,97,3.91,overcast clouds
16,16,Pacifica,US,37.6138,-122.4869,80.67,80,20,4.61,few clouds
19,19,Vaini,TO,-21.2000,-175.2000,71.76,100,90,8.05,moderate rain
23,23,Dingle,PH,10.9995,122.6711,75.83,91,81,8.55,broken clouds
26,26,Gunjur,GM,13.2019,-16.7339,82.47,78,100,4.61,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                243
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.74,overcast clouds,-23.1203,-134.9692,
2,Atar,MR,80.65,overcast clouds,20.5169,-13.0499,
9,Kudahuvadhoo,MV,81.88,overcast clouds,2.6708,72.8944,
10,Yeppoon,AU,78.58,scattered clouds,-23.1333,150.7333,
12,Aripuana,BR,86.31,overcast clouds,-9.1667,-60.6333,
13,Vanimo,PG,77.90,overcast clouds,-2.6741,141.3028,
16,Pacifica,US,80.67,few clouds,37.6138,-122.4869,
19,Vaini,TO,71.76,moderate rain,-21.2000,-175.2000,
23,Dingle,PH,75.83,broken clouds,10.9995,122.6711,
26,Gunjur,GM,82.47,overcast clouds,13.2019,-16.7339,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
#Needed numpy to perform blank to nan conversion method 
import numpy as np

In [24]:
#Replaced the blank hotel names with Nan in order to drop the rows 
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)

In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))